In [1]:
import numpy as np
import pandas as pd
import math
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.formula.api import glm
import collections
from statsmodels.sandbox.stats.multicomp import multipletests


In [2]:
df=pd.read_csv('Final_poisson.csv').iloc[:,1:]

In [3]:
import time

In [43]:
def conteo(newdf=df):
    conteos=newdf[['LAT_LON','SITIO_EXACTO_ACCIDENTE']].groupby(['LAT_LON','SITIO_EXACTO_ACCIDENTE']).size().reset_index(name='count')
    list_accidents=[]
    lat_lon=[]
    lat_lon_unico=np.unique(conteos['LAT_LON'])
    for i in range(len(lat_lon_unico)):
        a=conteos[conteos['LAT_LON']==lat_lon_unico[i]]
        suma=np.sum(a['count'])
        if np.max(a['count'])/suma>0.3:
            list_accidents.append(suma)
            lat_lon.append(lat_lon_unico[i])

    return(pd.DataFrame({'LAT_LON':lat_lon,'count':list_accidents}))
            

In [71]:
def conteo_perf(newdf=df):
    conteos=newdf[['LAT_LON','SITIO_EXACTO_ACCIDENTE']].groupby(['LAT_LON','SITIO_EXACTO_ACCIDENTE']).size().reset_index(name='count')
    grp_conteos = conteos.groupby('LAT_LON')['count'].agg(['sum','max'])
    grp_conteos['sum_rate'] = grp_conteos['max'] / grp_conteos['sum']
    filtered_conteos = grp_conteos[grp_conteos['sum_rate'] > 0.3]
    return_df = filtered_conteos.drop(['max', 'sum_rate'], axis=1)
    return_df.rename(columns = {'sum':'count'}, inplace = True) 
    return(return_df)

start_time = time.time()
b=conteo_perf()
print("--- %s seconds ---" % (time.time() - start_time))


--- 0.07230782508850098 seconds ---


In [45]:
start_time = time.time()
a=conteo()
print(a.sort_values(by=['count'], ascending=False))
print("--- %s seconds ---" % (time.time() - start_time))

                                    LAT_LON  count
5767                 11.0263731 -74.8037791     98
833   10.948341000000001 -74.77345809999998     93
1147  10.954438699999999 -74.83577079999998     87
760                  10.9468864 -74.7848518     86
5451          11.0145774 -74.82753829999999     76
...                                     ...    ...
2405          10.9755994 -74.78795840000002      1
2406         10.975601600000001 -74.7758872      1
2407         10.975605999999999 -74.8362139      1
2408         10.975613000000001 -74.7831058      1
5829                  11.043303 -74.8267913      1

[5830 rows x 2 columns]
--- 12.474770307540894 seconds ---


In [98]:
a.sort_values('count',ascending=False)

,LAT_LON,count
5767,11.0263731 -74.8037791,98
833,10.948341000000001 -74.77345809999998,93
1147,10.954438699999999 -74.83577079999998,87
760,10.9468864 -74.7848518,86
5451,11.0145774 -74.82753829999999,76
...,...,...
2405,10.9755994 -74.78795840000002,1
2406,10.975601600000001 -74.7758872,1
2407,10.975605999999999 -74.8362139,1
2408,10.975613000000001 -74.7831058,1
